# 神经网络  

神经网络可以使用torch.nn包来构建。

现在你对autograd自动求导有一些认识了，nn是构建在autograd之上的用来定义模型的使之呈现多样化。nn.Module包含一些层，forward(input)方法，返回一个output。

例子，一个图片分类器的网络

convnet卷积网络？

上图是一个简单的feed-forward前馈网络。它接受输入参数，给几个层一层层的feed数据，最后获得输出。

神经网络一个经典的训练过程是这样的：  
- 定义一个拥有一些可学习的参数(或者叫权重)的神经网络   
- 用输入数据集去迭代它
- 通过网络去处理输入数据  
- 计算损失（计算输出与正确值之间的差距）  
- 把梯度反向传播到网络的各参数中  
- 更新网络的权重，一般简单的使用规则: weight = weight - learning_rate * gradient

## 定义网络  

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        # 1 是输入的图片的通道，非RGB彩色， 6 输出通道是6(大概是6个卷积核吧)，3x3的卷积核
        self.conv1 = nn.Conv2d(1,6,3)
        self.conv2 = nn.Conv2d(6,16,3)
        # 仿射操作: y = Wx + b
        self.fc1 = nn.Linear(16*6*6,120)  # 6*6 来自于图片的维度
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
        
    def forward(self,x):
        # 通过一个(2,2)的窗口在做最大池
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        # 如果池化窗口是一个方块，那么你只需要填一个数字就行了，不需要像上面那样写个tuple
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    
    def num_flat_features(self,x):
        size = x.size()[1:]  # 除了batch批维度之外的所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


看上面，你必须定义一个forward函数，这个backward函数（就是梯度被计算的地方）是自动被定义的，不需要你自己定义，你可以用它来autograd自动求导。你可以在forward方法里面使用任何的Tensor操作。

我们可以通过net.parameters()返回来查看一个模型的可学习的参数。



In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1的weight

10
torch.Size([6, 1, 3, 3])
